In [1]:
from datasets import load_dataset
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from transformers import BartForConditionalGeneration, BartTokenizer, Trainer, TrainingArguments, DataCollatorForSeq2Seq
from sklearn.model_selection import train_test_split
warnings.filterwarnings("ignore")
import requests
from bs4 import BeautifulSoup 
import pandas as pd
from tqdm import tqdm

C:\Users\Akshay\anaconda3\envs\tf\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Context Based Dataset

In [29]:
dataset_1 = load_dataset("9wimu9/eli5_mult_answers_en_no_answer_in_context") # in deep
dataset_2 = load_dataset("mlxen/squad_1_1_smallcase_context") #one word
dataset_3 = load_dataset("nbtpj/multi-context-long-answer-dataset") # in short
dataset_8 = load_dataset("LasRuinasCirculares/sft_correct")
dataset_9 = load_dataset("robbiegwaldd/rephrase_train") # all

In [30]:
dataset_1

DatasetDict({
    train: Dataset({
        features: ['question', 'contexts', 'gold_answer'],
        num_rows: 71236
    })
    test: Dataset({
        features: ['question', 'contexts', 'gold_answer'],
        num_rows: 7916
    })
})

In [31]:
agent_1 = dataset_1["train"]["question"]
raw_context = dataset_1["train"]["contexts"]
modified_context = []
for c in raw_context:
    final_string = ""
    for i in c:
        final_string = final_string+" "+ i
    modified_context.append(final_string)
context = modified_context
agent_2 = dataset_1["train"]["gold_answer"]

In [32]:
agent_1.extend(dataset_2["train"]["question"])
context.extend(dataset_2["train"]["context"])
agent_2.extend([i["text"][0] for i in dataset_2["train"]["answers"]])

agent_1.extend([i.split("Question:")[1].replace("Answer:","") for i in dataset_8["train"]["instruction"]])
context.extend([i.replace("Evidence:","").replace("**","").split("Question:")[0].replace("\n","") for i in dataset_8["train"]["instruction"]])
agent_2.extend(dataset_8["train"]["output"])

In [33]:
channelNames =["pubmed_qa_X_squad_num_channel_1_test","pubmed_qa_X_squad_num_channel_1_train","pubmed_qa_X_squad_num_channel_2_test","pubmed_qa_X_squad_num_channel_2_train","pubmed_qa_X_squad_num_channel_3_test","pubmed_qa_X_squad_num_channel_3_train","pubmed_qa_X_squad_num_channel_4_test","pubmed_qa_X_squad_num_channel_4_train","pubmed_qa_num_channel_1_test","pubmed_qa_num_channel_1_train","pubmed_qa_num_channel_2_test","pubmed_qa_num_channel_2_train","pubmed_qa_num_channel_3_test","pubmed_qa_num_channel_3_train","pubmed_qa_num_channel_4_test","pubmed_qa_num_channel_4_train","squad_num_channel_1_test","squad_num_channel_1_train","squad_num_channel_2_test","squad_num_channel_2_train","squad_num_channel_3_test","squad_num_channel_3_train","squad_num_channel_4_test","squad_num_channel_4_train"]

In [34]:
question = []
cont = []
answer = []
for channel in channelNames:
    question.extend([i[0].split("<||||>")[0] for i in dataset_3[channel]["context"]])
    cont.extend([i[0].split("<||||>")[1] for i in dataset_3[channel]["context"]])
    answer.extend([i for i in dataset_3[channel]["answer"]])
temp_df = pd.DataFrame()
temp_df["question"] = question
temp_df["cont"] = cont
temp_df["answer"] = answer
df = temp_df.drop_duplicates(subset=['question','cont','answer'], keep='first')
agent_1.extend(df["question"].tolist())
context.extend(df["cont"].tolist())
agent_2.extend(df["answer"].tolist())

In [35]:
datasets = ["bart-base-squad.end2end.amazon","bart-base-squad.end2end.new_wiki","bart-base-squad.end2end.nyt","bart-base-squad.end2end.reddit","bart-base-squad.multitask.amazon","bart-base-squad.multitask.new_wiki","bart-base-squad.multitask.nyt","bart-base-squad.multitask.reddit","bart-base-squad.pipeline.amazon","bart-base-squad.pipeline.new_wiki","bart-base-squad.pipeline.nyt","bart-base-squad.pipeline.reddit","bart-base-squad.qg_reference.amazon","bart-base-squad.qg_reference.new_wiki","bart-base-squad.qg_reference.nyt","bart-base-squad.qg_reference.reddit","bart-large-squad.end2end.amazon","bart-large-squad.end2end.new_wiki","bart-large-squad.end2end.nyt","bart-large-squad.end2end.reddit","bart-large-squad.multitask.amazon","bart-large-squad.multitask.new_wiki","bart-large-squad.multitask.nyt","bart-large-squad.multitask.reddit","bart-large-squad.pipeline.amazon","bart-large-squad.pipeline.new_wiki","bart-large-squad.pipeline.nyt","bart-large-squad.pipeline.reddit","bart-large-squad.qg_reference.amazon","bart-large-squad.qg_reference.new_wiki","bart-large-squad.qg_reference.nyt","bart-large-squad.qg_reference.reddit","t5-base-squad.end2end.amazon","t5-base-squad.end2end.new_wiki","t5-base-squad.end2end.nyt","t5-base-squad.end2end.reddit","t5-base-squad.multitask.amazon","t5-base-squad.multitask.new_wiki","t5-base-squad.multitask.nyt","t5-base-squad.multitask.reddit","t5-base-squad.pipeline.amazon","t5-base-squad.pipeline.new_wiki","t5-base-squad.pipeline.nyt","t5-base-squad.pipeline.reddit","t5-base-squad.qg_reference.amazon","t5-base-squad.qg_reference.new_wiki","t5-base-squad.qg_reference.nyt","t5-base-squad.qg_reference.reddit","t5-large-squad.end2end.amazon","t5-large-squad.end2end.new_wiki"]

In [36]:
for i in tqdm(range(0,len(datasets))):
    # print(f'\rDataset Name: {dataset}', end='', flush=True)
    ds = load_dataset("lmqg/qa_squadshifts_synthetic", datasets[i])
    agent_1.extend(i for i in ds["train"]["question"])
    context.extend(i for i in ds["train"]["context"])
    agent_2.extend(i["text"][0] for i in ds["train"]["answers"])

100%|██████████| 50/50 [01:00<00:00,  1.21s/it]


In [37]:
dataset_9

DatasetDict({
    train: Dataset({
        features: ['id', 'original_snippet', 'rephrased_snippet'],
        num_rows: 291032
    })
})

In [38]:
len(agent_1)

1469571

In [39]:
questions = []
answers = []
conte = []
split_qa = [i.split("\n\n") for i in dataset_9["train"]["rephrased_snippet"]]
cont = [i.replace("\n","") for i in dataset_9["train"]["original_snippet"]]
for i in tqdm(range(0,len(split_qa))):
    q = [];a = [];c = []
    if "\nAnswer" not in split_qa[i][0]:
        if len(split_qa[i]) % 2 != 0:
            split_qa[i] = split_qa[i][:-1]
        for j in range(0,len(split_qa[i]),2):
            q.append(split_qa[i][j])
            a.append(split_qa[i][j+1])
            c.append(cont[i])
    else:
        new_split = [j.split("\n") for j in split_qa[i]]
        for j in range(0,len(new_split)):
            if len(new_split[j]) %2 !=0:
                continue
            q.append(new_split[j][0].replace("Question:",""))
            a.append(new_split[j][1].replace("Answer:",""))
            c.append(cont[i])
    answers.extend(a)
    questions.extend(q)
    conte.extend(c)
    # print(len(answers),len(questions))
    # print(split_qa[i])
agent_1.extend(questions)
context.extend(conte)
agent_2.extend(answers)

100%|██████████| 291032/291032 [00:00<00:00, 369566.84it/s]


In [40]:
len(agent_1)

1802143

In [41]:
len(context)

1802143

In [42]:
len(agent_2)

1802143

In [43]:
temp_df = pd.DataFrame()
temp_df["agent_1"] = agent_1
temp_df["context"] = context
temp_df["agent_2"] = agent_2

In [5]:
temp_df.to_csv("../Data/CleanedDatasets/ContextBasedQuestions.csv")

In [16]:
temp_df = pd.read_csv("../Data/CleanedDatasets/ContextBasedQuestions.csv")

In [40]:
temp_df.head()

,agent_2,agent_1
0,agent_2:These foods contain certain sugars tha...,<context> Actually it is because most of peop...
1,agent_2:I'd answer the question but the defaul...,<context> The decay of false vacuum is a fasc...
2,agent_2:Essentially the events in Crimea threa...,<context> National Sovereignty is a very big ...
3,agent_2:To have as little protection as is saf...,<context> Most helmets are pretty similar in ...
4,"agent_2:Your eyes are magnificent sensors, and...",<context> The reason is that a camera is basi...


In [18]:
temp_df["agent_2"] = "agent_2:"+temp_df["agent_2"]
temp_df["agent_1"] = "<context> " + temp_df["context"] + "agent_1:"+temp_df["old_agent_1"]

In [19]:
temp_df.dropna(inplace = True)
temp_df.reset_index(inplace=True)

In [36]:
import math
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
agent_1_extended = []
agent_2_extended = []
tokenizedLengthList = []
idx = 0
for input_text in tqdm(temp_df["agent_1"].to_list()):
    try:
        input_tokenized = tokenizer(input_text,return_tensors='pt')
        tokenizedLength = len(input_tokenized["input_ids"][0])
        tokenizedLengthList.append(tokenizedLength)
        if tokenizedLength < 510:
            agent_1_extended.append(input_text)
            agent_2_extended.append(temp_df["agent_2"][idx])
    except Exception as e:
        print(e)
    idx = idx+1
    # if tokenizedLength > 510 and tokenizedLength < 1500:
        
    #         num_of_loops = math.ceil(tokenizedLength/510)
    #         slice_length = math.ceil(len(input_text)/num_of_loops)
    #         for i in range(0,num_of_loops):
    #             if i == 0:
    #                 agent_1_extended.append("agent_1:"+(input_text[i*slice_length:(i*slice_length)+slice_length])+"<question>"+temp_df["old_agent_1"][idx])
    #             else:
    #                 agent_1_extended.append("agent_1:"+"<context>"+(input_text[(i*slice_length)-50:(i*slice_length)+slice_length])+"<question>"+temp_df["old_agent_1"][idx])
    
    # else:
    #     agent_1_extended.append("agent_1:"+input_text)

100%|██████████| 1799426/1799426 [45:44<00:00, 655.72it/s] 


In [37]:
len(agent_1_extended)

1400831

In [38]:
len(agent_2_extended)

1400831

In [39]:
temp_df = pd.DataFrame()
temp_df["agent_2"] =agent_2_extended
temp_df["agent_1"] =agent_1_extended

In [11]:
agent_1_extended[2]

'<context>  National Sovereignty is a very big deal so anytime a country (especially a country with nukes) starts to reject previously agreed upon borders it is a big deal.\n\nThe reason you should care is that Russia has been for quite sometime trying to maintain some degree of influence over several former Soviet republics.  As some of these states move away from Russia, it is going to cause conflict and potentially a full blown war. likely to evolve into nothing\n\ncrimea was granted to ukraine as a restitution gift and well over half the population is in favour of annexation\n\nthey already speak russian and maintain russian culture\n\nputin is a total dick but once this blows over they will pay lower taxes to an equally corrupt government and receive additional social benefits\n\nyeah, its a huge problem that a part of a country is essentially being taken over but it hardly sets precedent for speculations of a third world war\n\nagent_1:what is the big deal about russia invading u

In [55]:
temp_df.to_csv("../Data/CleanedDatasets/ContextBasedQuestions.csv")

In [53]:
temp_df["agent_1"][1400700]

"<context> 5 best Java seeds for Minecraft 1.19 updateThe 1.19 update for Minecraft introduced many new features. \xa0Two major biomes, Deep Dark and Mangrove Swamp, have been added to the game and are quickly becoming quite popular.The two biomes are arguably the biggest addition to the game. \xa0Deep Dark and Mangrove Swamp both provide reasons to visit them, but there's only one problem: they can be difficult to find.Minecraft seeds to try in Java Edition version 1.19Multiple Blue Rings\xa0Note: The 1.18 update introduced seed universality, so seeds will work on both versions with minimal differences. \xa0The main differences are when moving from Java to Bedrock, but not vice versa.The ancient city in 1.19 is a new but extremely rare structure. \xa0It's a challenge to trace a regular Deep Dark with just the Skulk and the Warden. \xa0Most of those found do not even contain any ancient cities.Seed: -457009213479927390This seed puts players into an amazing spawn. \xa0Gamers come right 

In [54]:
temp_df["agent_2"][1400700]

'agent_2: In Minecraft 1.19, there are two new biomes added: "Deep Dark" and "Mangrove Swamp". The Deep Dark biome provides reasons to visit it but can be difficult to find due to its location near a swamp. Meanwhile, the Mangrove Swamp biome has only one problem - it can be difficult to find as well. Minecraft seeds like -457009213479927390, -6709148406763899126, and -2110863992403414331 can be used to trace a Deep Dark with just the Skulk and Warden. These seeds contain good loot but do not necessarily have Mangrove Swamp as they are located in swamps instead of in the swamp biome. The seeds -2110863992403414331 and 1450778142214593647 can be used to find Mangrove Swamp by using a village as an alternative spawn point. The seeds -6709148406763899126 and 6705'

### Google Search

In [24]:
from googlesearch import search
def get_google_search_links(query):
    return [link for link in search(query)]

#### Normal Search Queries

In [62]:
website_links = get_google_search_links('what is backtesting in algorithmic trading')

In [63]:
website_links

['https://dataman-ai.medium.com/practical-algorithmic-trading-2-backtesting-19971c8a89bd',
 'https://www.utradealgos.com/blog/what-is-algo-backtesting-and-how-does-it-work',
 'https://medium.com/@anrodz/algo-trading-backtesting-your-algorithm-bd6d7385c89c',
 'https://speedbot.tech/blog/speedbot-1/future-of-backtesting-in-algo-trading-what-to-know-41',
 'https://www.marketfeed.com/read/en/the-ultimate-guide-to-backtesting-algo-trading-strategies',
 'https://www.investopedia.com/terms/b/backtesting.asp',
 'https://www.buildalpha.com/backtesting-trading-strategies/',
 'https://tradetron.tech/blog/backtest/',
 'https://zorro-project.com/backtest.php',
 'https://blog.quantinsti.com/backtesting/',
 'https://zorro-project.com/backtest.php',
 'https://utradealgos.com/blog/why-algo-backtesting-is-crucial-for-algorithmic-trading/']

In [66]:
if ".gov" not in website_links[1]:
    URL = website_links[1] 
    r = requests.get(URL) 
    soup = BeautifulSoup(r.content, 'html5lib')
else:
    print("Caution Govt website")

In [67]:
raw_content = soup.find_all(["p","h"])
filtered_content = [i.text for i in raw_content]
filtered_content

['Algo backtesting is a systematic process that allows traders to assess the performance of their trading strategies using historical data. By simulating past market conditions and applying their trading rules to historical price and volume data, traders can gain valuable insights into how their strategies would have fared in the real world. It involves the following fundamental steps: ',
 'Traders and developers meticulously craft a set of rules and conditions that constitute the trading strategy. These rules encompass a variety of aspects:',
 'Traders need to obtain reliable historical data for the financial assets they intend to trade, such as stocks, commodities, or forex pairs. Ensuring the historical data is clean and accurate is important, as poor data quality can distort backtesting results. This is where platforms like uTrade Algos become important as they provide reliable historical data. ',
 "After the simulation, traders perform a comprehensive analysis of the strategy's hi

#### Reasearch Paper Queries

In [198]:
website_links = get_google_search_links('what is backtesting in algorithmic trading')

In [199]:
website_links

['https://medium.com/@anrodz/algo-trading-backtesting-your-algorithm-bd6d7385c89c',
 'https://www.investopedia.com/terms/b/backtesting.asp',
 'https://utradealgos.com/blog/what-is-algo-backtesting-and-how-does-it-work/',
 'https://speedbot.tech/blog/speedbot-1/future-of-backtesting-in-algo-trading-what-to-know-41',
 'https://dataman-ai.medium.com/practical-algorithmic-trading-2-backtesting-19971c8a89bd',
 'https://zorro-project.com/backtest.php',
 'https://blog.quantinsti.com/backtesting/',
 'https://www.linkedin.com/pulse/what-backtesting-why-important-algorithmic-trading-algoedge-cad-lcp7e',
 'https://www.udemy.com/course/mastering-backtesting-for-algorithmic-trading/',
 'https://www.linkedin.com/pulse/what-backtesting-why-important-algorithmic-trading-algoedge-cad-lcp7e',
 'https://www.udemy.com/course/mastering-backtesting-for-algorithmic-trading/',
 'https://tradetron.tech/blog/backtest/']

In [200]:
URL = website_links[2] 
r = requests.get(URL) 
soup = BeautifulSoup(r.content, 'html5lib')
raw_content = soup.find_all(["p","h"])
filtered_content = [i.text for i in raw_content]
filtered_content

['uTrade Algos',
 'Home » Blogs » What Is Algo Backtesting and How Does It Work?',
 'In the fast-paced world of financial markets, traders and investors are constantly seeking an edge to stay ahead of the curve. Algorithmic trading, or algo trading, has emerged as a powerful tool to gain this competitive advantage. It involves the use of automated trading systems, often fuelled by complex algorithms, to execute trades with speed and precision. However, before deploying these algorithms in live markets, algo traders employ a crucial step to evaluate and fine-tune their strategies: backtesting. Here, we will find out what algo backtesting is and how it works.\xa0',
 'Table of Contents',
 'Algo backtesting is a systematic process that allows traders to assess the performance of their trading strategies using historical data. By simulating past market conditions and applying their trading rules to historical price and volume data, traders can gain valuable insights into how their strategie